In [90]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from operator import itemgetter

os.chdir("/home/william/Desktop")

In [91]:
class Analisis_Predictivo:

    def __init__(self, predecir=None, predictoras = [],
                 modelo = None,train_size = None,random_state = None,
                 xtrain=None, ytrain=None, xtest=None, ytest=None,datos = None):
        '''
        datos: Datos completos y listos para construir un modelo

        modelo: Instancia de una Clase de un método de clasificación(KNN,Árboles,SVM,etc).
        Si no especifica un modelo no podrá utilizar el método fit_n_review()

        predecir: Nombre de la variable a predecir

        predictoras: Lista de los nombres de las variables predictoras.
        Si vacío entonces utiliza todas las variables presentes excepto la variable a predecir.

        train_size: Proporción de la tabla de entrenamiento respecto a la original.

        random_state: Semilla aleatoria para la división de datos(training-testing).
        '''
        self.datos = datos
        self.predecir = predecir
        self.predictoras = predictoras
        self.modelo = modelo
        self.random_state = random_state
        self.X_train = xtrain
        self.X_test = xtest
        self.y_train = ytrain
        self.y_test = ytest
        if modelo != None and train_size != None:
            self.train_size = train_size
            self._training_testing()


    def _training_testing(self):
        if len(self.predictoras) == 0:
            X = self.datos.drop(columns=[self.predecir])
        else:
            X = self.datos[self.predictoras]

        y = self.datos[self.predecir].values

        train_test = train_test_split(X, y, train_size=self.train_size,
                                      random_state=self.random_state)
        self.X_train, self.X_test,self.y_train, self.y_test = train_test


    def fit_predict(self):
        if self.modelo != None:
            self.modelo.fit(self.X_train,self.y_train)
            return self.modelo.predict(self.X_test)

    def fit_predict_resultados(self, imprimir = True, inyectado=None, custom=False):
        if(self.modelo != None):
            if custom == False:
                y = self.datos[self.predecir].values
            else:
                y= inyectado
            prediccion = self.fit_predict()
            MC = confusion_matrix(self.y_test, prediccion)
            indices = self.indices_general(MC,list(np.unique(y)))
            if imprimir == True:
                for k in indices:
                    print("\n%s:\n%s"%(k,str(indices[k])))

            return indices

    def indices_general(self,MC, nombres = None):
        "Método para calcular los índices de calidad de la predicción"
        precision_global = np.sum(MC.diagonal()) / np.sum(MC)
        error_global = 1 - precision_global
        precision_categoria  = pd.DataFrame(MC.diagonal()/np.sum(MC,axis = 1)).T
        if nombres!=None:
            precision_categoria.columns = nombres
        return {"Matriz de Confusión":MC,
                "Precisión Global":precision_global,
                "Error Global":error_global,
                "Precisión por categoría":precision_categoria}

    def distribucion_variable_predecir(self):
        "Método para graficar la distribución de la variable a predecir"
        variable_predict = self.predecir
        data = self.datos
        colors = list(dict(**mcolors.CSS4_COLORS))
        df = pd.crosstab(index=data[variable_predict],columns="valor") / data[variable_predict].count()
        fig = plt.figure(figsize=(10,9))
        g = fig.add_subplot(111)
        countv = 0
        titulo = "Distribución de la variable %s" % variable_predict
        for i in range(df.shape[0]):
            g.barh(1,df.iloc[i],left = countv, align='center',color=colors[11+i],label= df.iloc[i].name)
            countv = countv + df.iloc[i]
        vals = g.get_xticks()
        g.set_xlim(0,1)
        g.set_yticklabels("")
        g.set_title(titulo)
        g.set_ylabel(variable_predict)
        g.set_xticklabels(['{:.0%}'.format(x) for x in vals])
        countv = 0
        for v in df.iloc[:,0]:
            g.text(np.mean([countv,countv+v]) - 0.03, 1 , '{:.1%}'.format(v), color='black', fontweight='bold')
            countv = countv + v
        g.legend(loc='upper center', bbox_to_anchor=(1.08, 1), shadow=True, ncol=1)

    def poder_predictivo_categorica(self, var:str):
        "Método para ver la distribución de una variable categórica respecto a la predecir"
        data = self.datos
        variable_predict = self.predecir
        df = pd.crosstab(index= data[var],columns=data[variable_predict])
        df = df.div(df.sum(axis=1),axis=0)
        titulo = "Distribución de la variable %s según la variable %s" % (var,variable_predict)
        g = df.plot(kind='barh',stacked=True,legend = True, figsize = (10,9), \
                    xlim = (0,1),title = titulo, width = 0.8)
        vals = g.get_xticks()
        g.set_xticklabels(['{:.0%}'.format(x) for x in vals])
        g.legend(loc='upper center', bbox_to_anchor=(1.08, 1), shadow=True, ncol=1)
        for bars in g.containers:
            plt.setp(bars, width=.9)
        for i in range(df.shape[0]):
            countv = 0
            for v in df.iloc[i]:
                g.text(np.mean([countv,countv+v]) - 0.03, i , '{:.1%}'.format(v), color='black', fontweight='bold')
                countv = countv + v


    def poder_predictivo_numerica(self,var:str):
        "Función para ver la distribución de una variable numérica respecto a la predecir"
        sns.FacetGrid(self.datos, hue=self.predecir, height=6).map(sns.kdeplot, var, shade=True).add_legend()


In [92]:
def transformToDataFrame(modelo):
    resultados = modelo.fit_predict_resultados(imprimir=True)
    precisionGlobal = pd.DataFrame({"Precisión Global" : [resultados['Precisión Global']]})
    errorGlobal = pd.DataFrame({"Error Global" : [resultados['Error Global']]})
    precisonCategoria = resultados['Precisión por categoría']
    finalDataFrame = precisionGlobal.join(errorGlobal).join(precisonCategoria)
    return finalDataFrame

In [93]:
potabilidad = pd.read_csv("potabilidad_V2.csv")
potabilidad = potabilidad.iloc[:,1:]
potabilidad["Potability"] = potabilidad["Potability"].astype('category')

El primer modelo sin estandarizar los datos  muestra unos resultados bastante malos, por que como podemos ver en el dataframe de abajo, aunque tiene una precision global interesante, la verdadera metrica, se ve en el "NO" y el "SI", y podemos ver que el valor de si es muy bajo

In [94]:
modelo = MLPClassifier(hidden_layer_sizes= (6,) , activation= "identity", solver="adam", random_state=0)
analisisModelo1 = Analisis_Predictivo(datos=potabilidad,predecir="Potability",modelo=modelo,
                                       train_size= 0.75, random_state=0)
transformToDataFrame(analisisModelo1)


Matriz de Confusión:
[[257  52]
 [161  33]]

Precisión Global:
0.5765407554671969

Error Global:
0.42345924453280315

Precisión por categoría:
         No        Si
0  0.831715  0.170103


,Precisión Global,Error Global,No,Si
0,0.576541,0.423459,0.831715,0.170103


Al igual que el primer modelo sin estandarizar los datos muestra unos resultados pesimos, el segundo modelo muestra unos resultados aun peores que el anterior, aunque parece que predice bien el "no", esto no es cierto por que al ver el valor del "SI",  podemos que lo que sucede es que categoriza casi todo como "NO", entonces por eso tiene ese valor tan alto

In [95]:
modelo2 = MLPClassifier(hidden_layer_sizes= (10,25) , activation="tanh", solver="sgd", random_state=0)
analisisModelo2 = Analisis_Predictivo(datos=potabilidad,predecir= "Potability",modelo=modelo2,
                                       train_size= 0.75, random_state=0)
transformToDataFrame(analisisModelo2)


Matriz de Confusión:
[[309   0]
 [192   2]]

Precisión Global:
0.6182902584493042

Error Global:
0.3817097415506958

Precisión por categoría:
    No        Si
0  1.0  0.010309


,Precisión Global,Error Global,No,Si
0,0.61829,0.38171,1.0,0.010309


In [96]:
potabilidad.iloc[:,:-1] = StandardScaler().fit_transform(potabilidad.iloc[:,:-1])

Estas metricas pertenecen al modelo 1, con la estandarizacion de datos y aunque los resultados son malo al igual que en el anterior de acuerdo a la matriz de confusion podemos darnos cuenta, que realmente si mejoran bastante, con respecto al anterior

In [97]:
modelo = MLPClassifier(hidden_layer_sizes= (6,) , activation= "identity", solver="adam", random_state=0)
analisisModelo1 = Analisis_Predictivo(datos=potabilidad,predecir= "Potability",modelo=modelo,
                                       train_size= 0.75, random_state=0)

transformToDataFrame(analisisModelo1)



Matriz de Confusión:
[[304   5]
 [184  10]]

Precisión Global:
0.6242544731610338

Error Global:
0.3757455268389662

Precisión por categoría:
         No        Si
0  0.983819  0.051546


,Precisión Global,Error Global,No,Si
0,0.624254,0.375746,0.983819,0.051546


En el caso del modelo 2 podemos ver que se mejora en el tema del si, aunque claramente se puede notar que es muy poco, si nota que la estadarizacion de datos es demasiado importante cuando estamos usando las redes neuronales

In [98]:
modelo2 = MLPClassifier(hidden_layer_sizes= (10,25) , activation="tanh", solver="sgd", random_state=0)
analisisModelo2 = Analisis_Predictivo(datos=potabilidad,predecir= "Potability",modelo=modelo2,
                                       train_size= 0.75, random_state=0)
transformToDataFrame(analisisModelo2)


Matriz de Confusión:
[[306   3]
 [192   2]]

Precisión Global:
0.6123260437375746

Error Global:
0.3876739562624254

Precisión por categoría:
         No        Si
0  0.990291  0.010309


,Precisión Global,Error Global,No,Si
0,0.612326,0.387674,0.990291,0.010309


In [99]:
wine = pd.read_csv("wine.csv")
wine = wine.iloc[:,1:]
wine

,volatil.acidez,citrica.acidez,residual.azucar,cloruros,libre.sulfuro.dioxido,total.sulfuro.dioxido,densidad,pH,sulfitos,alcohol,calidad,tipo
0,0.30,0.21,1.1,0.032,31.0,111.0,0.98890,2.97,0.42,12.2,6,blanco
1,0.61,0.01,1.9,0.080,8.0,25.0,0.99746,3.69,0.73,10.5,5,tinto
2,0.32,0.23,16.2,0.055,36.0,176.0,0.99860,3.26,0.54,9.1,5,blanco
3,0.22,0.29,8.9,0.050,24.0,90.0,0.99556,3.29,0.46,9.8,6,blanco
4,0.43,0.27,1.1,0.054,5.0,110.0,0.99390,3.24,0.52,9.1,4,blanco
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,blanco
6493,0.26,0.43,12.6,0.033,64.0,230.0,0.99740,3.08,0.38,8.9,5,blanco
6494,0.25,0.48,10.0,0.044,78.0,240.0,0.99655,3.25,0.47,9.5,6,blanco
6495,0.60,0.20,9.9,0.070,21.0,174.0,0.99710,3.03,0.54,9.1,5,blanco


Para el primer modelo , la tabla de vinos nos envia unos resultados sorprendentes, podemos ver que aun sin estandarizacion tenemos un resultado del 97% en la precision global, pero igualmente se puede ver en los valores especificos de si es tinto o blanco, podemos ver que es super alto, lo que a su vez indica que el error global es muy bajo.

In [100]:
modelo = MLPClassifier(hidden_layer_sizes= (6,) , activation= "identity", solver="adam", random_state=0)
analisisModelo1 = Analisis_Predictivo(datos=wine,predecir= "tipo",modelo=modelo,
                                       train_size= 0.80, random_state=0)
transformToDataFrame(analisisModelo1)


Matriz de Confusión:
[[965  12]
 [ 18 305]]

Precisión Global:
0.9769230769230769

Error Global:
0.023076923076923106

Precisión por categoría:
     blanco     tinto
0  0.987718  0.944272


/home/william/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,Precisión Global,Error Global,blanco,tinto
0,0.976923,0.023077,0.987718,0.944272


Para el segundo modelo , la tabla de vinos nos envia unos resultados sorprendentes, podemos ver que aun sin estandarizacion tenemos un resultado del 92% en la precision global, pero igualmente se puede ver en los valores especificos de si es tinto o blanco, podemos ver que es super alto, lo que a su vez indica que el error global es muy bajo.

In [101]:
modelo2 = MLPClassifier(hidden_layer_sizes= (10,25) , activation="tanh", solver="sgd", random_state=0)
analisisModelo2 = Analisis_Predictivo(datos=wine,predecir= "tipo",modelo=modelo2,
                                       train_size= 0.80, random_state=0)
transformToDataFrame(analisisModelo2)


Matriz de Confusión:
[[936  41]
 [ 59 264]]

Precisión Global:
0.9230769230769231

Error Global:
0.07692307692307687

Precisión por categoría:
     blanco     tinto
0  0.958035  0.817337


,Precisión Global,Error Global,blanco,tinto
0,0.923077,0.076923,0.958035,0.817337


In [102]:
wine.iloc[:,:-1] = StandardScaler().fit_transform(wine.iloc[:,:-1])

En el caso de cuando es estandarizado, lo que sucede es que mejora bastante con respecto al modelo sin estandarizar, que de igualmanera era bastante alta, pero ahora esta espectacular

In [103]:
modelo = MLPClassifier(hidden_layer_sizes= (6,) , activation= "identity", solver="adam", random_state=0)
analisisModelo1 = Analisis_Predictivo(datos=wine,predecir= "tipo",modelo=modelo,
                                       train_size= 0.80, random_state=0)

transformToDataFrame(analisisModelo1)


Matriz de Confusión:
[[974   3]
 [  4 319]]

Precisión Global:
0.9946153846153846

Error Global:
0.005384615384615432

Precisión por categoría:
     blanco     tinto
0  0.996929  0.987616


,Precisión Global,Error Global,blanco,tinto
0,0.994615,0.005385,0.996929,0.987616


De igualmanera para el modelo 2 estandarizado, aunque el modelo 2 sin estandarizar fue bueno con un 92%, este modelo lo supera por mucho.

Realmente la conclusion que me dejo este ejercicio es que siempre en las redes neuronales se obtiene un mejor resultado cuando se realiza la estandarizacion de datos.

In [104]:
modelo2 = MLPClassifier(hidden_layer_sizes= (10,25) , activation="tanh", solver="sgd", random_state=0)
analisisModelo2 = Analisis_Predictivo(datos=wine,predecir= "tipo",modelo=modelo2,
                                      train_size= 0.80, random_state=0)
transformToDataFrame(analisisModelo2)



Matriz de Confusión:
[[974   3]
 [  5 318]]

Precisión Global:
0.9938461538461538

Error Global:
0.006153846153846176

Precisión por categoría:
     blanco    tinto
0  0.996929  0.98452


,Precisión Global,Error Global,blanco,tinto
0,0.993846,0.006154,0.996929,0.98452


In [105]:
dataTest = pd.read_csv("ZipDataTestCod.csv", sep=";")
dataTrain = pd.read_csv("ZipDataTrainCod.csv", sep=";")

In [106]:
def comprimir(data, p=4, filtradoPorMayor = False):
  t = pd.DataFrame()

  for k in range(len(data)):
    matriz = data.iloc[k]
    matriz = np.array([matriz]).reshape(16,16)
    inicial_espe = 0
    fin_espe = p
    contador = 0
    datos = []

    for i in range(p):
      inicial = 0
      fin = p

      for j in range(p):
        mayor = matriz[inicial_espe:fin_espe, inicial:fin]
        num = max(map(max, mayor))

        if not filtradoPorMayor:
            datos.append(sum(sum(matriz[inicial_espe:fin_espe, inicial:fin]))/(p*p))
        else:
            datos.append(num)

        contador += 1
        inicial += p
        fin += p

      inicial_espe += p
      fin_espe += p

    t.loc[:,k] = datos

  return t.T

In [107]:
ytest = dataTest['Numero']
dataTest = dataTest.drop(columns=['Numero'])

In [108]:
ytrain = dataTrain['Numero']
dataTrain = dataTrain.drop(columns=['Numero'])

In [109]:
modeloDataSinComprimir = MLPClassifier(hidden_layer_sizes =(250,100,50,25), max_iter = 50000, activation = 'relu', solver = 'adam',random_state=0)
analisisModelo2 = Analisis_Predictivo(predecir= "Numero",modelo=modeloDataSinComprimir, random_state=0, xtrain= dataTrain, xtest=dataTest, ytrain=ytrain, ytest=ytest)
analisisModelo2.fit_predict_resultados(inyectado=ytrain.append(ytest), custom=True)


Matriz de Confusión:
[[352   0   3   2   1   0   1   0   0   0]
 [  3 147   0   1   3   1   0   0   5   0]
 [  1   2 187   2   4   0   2   0   0   2]
 [  3   0   2 181   0   6   1   2   3   0]
 [  0   1   6   0 169   0   0   1   0   0]
 [  4   1   3   2   1 150   0   1   4   0]
 [  0   1   2   3   0   1 163   0   0   0]
 [  0   0   7   1   2   1   0 136   0   0]
 [  1   9   0   3   0   4   0   0 149   0]
 [  0   0   4   1   1   1   3   0   1 253]]

Precisión Global:
0.9402092675635276

Error Global:
0.05979073243647237

Precisión por categoría:
       cero    cinco  cuatro       dos     nueve      ocho      seis    siete  \
0  0.980501  0.91875   0.935  0.914141  0.954802  0.903614  0.958824  0.92517   

      tres       uno  
0  0.89759  0.958333  


{'Matriz de Confusión': array([[352,   0,   3,   2,   1,   0,   1,   0,   0,   0],
        [  3, 147,   0,   1,   3,   1,   0,   0,   5,   0],
        [  1,   2, 187,   2,   4,   0,   2,   0,   0,   2],
        [  3,   0,   2, 181,   0,   6,   1,   2,   3,   0],
        [  0,   1,   6,   0, 169,   0,   0,   1,   0,   0],
        [  4,   1,   3,   2,   1, 150,   0,   1,   4,   0],
        [  0,   1,   2,   3,   0,   1, 163,   0,   0,   0],
        [  0,   0,   7,   1,   2,   1,   0, 136,   0,   0],
        [  1,   9,   0,   3,   0,   4,   0,   0, 149,   0],
        [  0,   0,   4,   1,   1,   1,   3,   0,   1, 253]]),
 'Precisión Global': 0.9402092675635276,
 'Error Global': 0.05979073243647237,
 'Precisión por categoría':        cero    cinco  cuatro       dos     nueve      ocho      seis    siete  \
 0  0.980501  0.91875   0.935  0.914141  0.954802  0.903614  0.958824  0.92517   
 
       tres       uno  
 0  0.89759  0.958333  }

In [110]:
dataTrain

,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,...,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257
0,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.000,-0.631,0.862,-0.167,...,0.304,0.823,1.000,0.482,-0.474,-0.991,-1.000,-1.000,-1.000,-1.0
1,-1.0,-1.0,-1.0,-0.813,-0.671,-0.809,-0.887,-0.671,-0.853,-1.000,...,-0.671,-0.671,-0.033,0.761,0.762,0.126,-0.095,-0.671,-0.828,-1.0
2,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-0.996,...,-1.000,-1.000,-1.000,-0.109,1.000,-0.179,-1.000,-1.000,-1.000,-1.0
3,-1.0,-1.0,-1.0,-1.000,-1.000,-0.273,0.684,0.960,0.450,-0.067,...,-0.318,1.000,0.536,-0.987,-1.000,-1.000,-1.000,-1.000,-1.000,-1.0
4,-1.0,-1.0,-1.0,-1.000,-1.000,-0.928,-0.204,0.751,0.466,0.234,...,0.466,0.639,1.000,1.000,0.791,0.439,-0.199,-0.883,-1.000,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,-1.0,-1.0,-1.0,-0.988,-0.527,-0.208,0.620,1.000,0.467,0.396,...,-0.116,0.899,0.416,-0.510,-1.000,-1.000,-1.000,-1.000,-1.000,-1.0
7287,-1.0,-1.0,-1.0,-0.990,0.708,0.557,0.347,-0.107,-0.758,-0.975,...,0.697,0.636,0.167,-0.968,-1.000,-1.000,-1.000,-1.000,-1.000,-1.0
7288,-1.0,-1.0,-1.0,-0.783,-0.984,-0.827,0.068,1.000,1.000,1.000,...,0.805,1.000,1.000,0.727,-0.342,-0.933,-1.000,-1.000,-1.000,-1.0
7289,-1.0,-1.0,-1.0,-1.000,-1.000,-0.549,0.463,0.999,0.999,0.999,...,-0.231,0.621,0.999,-0.042,-0.231,-0.687,-1.000,-1.000,-1.000,-1.0


In [111]:
dataTrainComprimido = comprimir(dataTrain)
dataTrainComprimido

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-1.000000,-0.474625,-0.375937,-1.000000,-0.821563,0.193125,-0.717812,-0.804562,-0.008250,-0.009562,0.027188,-0.051063,-0.576125,0.692063,0.627938,-0.785875
1,-0.859687,0.531687,0.310500,-0.038938,-0.833000,0.475688,0.214625,-0.871188,-0.000250,-0.895750,-0.811813,0.244750,-0.182938,0.228562,0.386750,0.313188
2,-1.000000,-0.987500,-0.138437,-0.937500,-0.733625,-0.354875,0.047500,-0.998563,-0.616125,0.111375,0.332375,-0.601938,-1.000000,-0.900688,0.170063,-1.000000
3,-0.869625,0.109812,0.074313,-0.647687,-0.667375,-0.701562,-0.093875,-0.848375,-1.000000,-0.911438,-0.075312,-1.000000,-1.000000,-0.403062,-0.557000,-1.000000
4,-1.000000,0.025062,0.063687,-1.000000,-1.000000,-0.238062,0.320750,-0.984812,-0.895563,-0.882187,-0.735687,-0.188812,-0.203000,0.169000,0.037375,-0.145812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,-0.854437,0.637375,0.357000,-0.967375,-1.000000,-0.767750,0.365125,-0.996875,-0.955375,-0.309562,0.132625,-0.757437,-0.880563,0.320063,0.131625,-0.976750
7287,-0.981437,0.179000,-0.028625,-1.000000,-1.000000,-0.686937,0.449500,-1.000000,-1.000000,-0.681312,0.398062,-0.960938,-0.992063,0.413250,-0.019063,-1.000000
7288,-0.512062,0.561813,0.824500,-0.637750,-0.980313,-0.647375,0.697625,-0.761813,-0.863438,-0.600250,0.155438,-0.295813,-0.785125,0.599250,0.676125,-0.832250
7289,-0.897938,0.652687,0.769000,-0.689937,0.051688,0.787000,0.502938,0.386000,0.496375,0.126250,-0.066937,0.603812,-0.626812,0.627125,0.731625,-0.330000


In [112]:
dataTestComprimido = comprimir(dataTest)
dataTestComprimido

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.868688,0.056500,-0.001625,-0.641938,-0.152250,-0.787312,0.184125,-0.451750,-0.807500,-0.135188,0.297625,-1.000000,-1.000000,-0.922813,-0.059500,-1.000000
1,-0.966688,-0.569750,-1.000000,-1.000000,-0.187875,-0.763562,-0.796687,-0.183313,0.000500,-0.785813,0.180438,0.031688,-0.368125,-0.036375,-0.170687,-0.899500
2,-0.483062,0.198125,0.123875,-0.561312,-1.000000,-0.696688,0.269875,-0.489875,-1.000000,-0.404500,-0.152937,-0.539875,-0.471187,0.135438,0.294250,-0.472500
3,-0.992687,-0.595375,-1.000000,-1.000000,-0.011312,-0.623188,-0.505563,-0.006125,0.172937,-0.763313,0.045813,-0.019875,-0.626375,-0.320437,-0.358437,-0.920875
4,-1.000000,-0.238187,-0.378500,-1.000000,-0.222188,-0.406188,-1.000000,-1.000000,0.177125,-0.787312,0.076625,0.428437,0.231250,0.520563,0.112063,-0.462875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,-1.000000,0.180188,-0.055812,-1.000000,-1.000000,-0.981250,0.283312,-1.000000,-1.000000,-1.000000,-0.018750,-0.735563,-0.652750,0.177312,0.208250,-0.794000
2003,-1.000000,0.063562,0.408312,-0.901750,-0.892313,0.577125,0.548500,-0.991187,-1.000000,-0.785375,0.246250,-1.000000,-1.000000,-0.592750,-0.196375,-1.000000
2004,-0.810187,-0.005563,-0.743188,-1.000000,-0.096000,-0.138250,0.778250,-0.296687,-0.415937,0.480875,-0.358563,-1.000000,-1.000000,0.290500,-0.801813,-1.000000
2005,-0.624563,0.787875,0.270062,-0.873812,0.312312,-0.152375,-0.207500,0.167500,0.440250,-0.197437,0.257563,0.159500,-0.500687,0.853250,0.471125,-0.844000


In [113]:
modeloDataSinComprimir = MLPClassifier(hidden_layer_sizes =(250,100,50,25), max_iter = 50000, activation = 'relu', solver = 'adam',random_state=0)
analisisModelo2 = Analisis_Predictivo(predecir= "Numero",modelo=modeloDataSinComprimir, random_state=0, xtrain= dataTrainComprimido, xtest=dataTestComprimido, ytrain=ytrain, ytest=ytest)
analisisModelo2.fit_predict_resultados(inyectado=ytrain.append(ytest), custom=True)


Matriz de Confusión:
[[343   1   1   2   0   6   2   2   0   2]
 [  2 145   1   0   3   2   1   0   6   0]
 [  0   1 179   2  12   0   4   1   0   1]
 [  4   5   2 175   1   5   2   1   3   0]
 [  0   0   3   0 170   1   0   3   0   0]
 [  3   2   2   1   6 145   0   0   5   2]
 [  8   1   2   1   0   0 157   0   0   1]
 [  0   0   4   1   7   1   0 131   3   0]
 [  3  17   0   2   2   7   0   0 134   1]
 [  1   1   3   1   0   5   3   2   1 247]]

Precisión Global:
0.9098156452416543

Error Global:
0.09018435475834574

Precisión por categoría:
       cero    cinco  cuatro       dos     nueve      ocho      seis  \
0  0.955432  0.90625   0.895  0.883838  0.960452  0.873494  0.923529   

      siete      tres       uno  
0  0.891156  0.807229  0.935606  


{'Matriz de Confusión': array([[343,   1,   1,   2,   0,   6,   2,   2,   0,   2],
        [  2, 145,   1,   0,   3,   2,   1,   0,   6,   0],
        [  0,   1, 179,   2,  12,   0,   4,   1,   0,   1],
        [  4,   5,   2, 175,   1,   5,   2,   1,   3,   0],
        [  0,   0,   3,   0, 170,   1,   0,   3,   0,   0],
        [  3,   2,   2,   1,   6, 145,   0,   0,   5,   2],
        [  8,   1,   2,   1,   0,   0, 157,   0,   0,   1],
        [  0,   0,   4,   1,   7,   1,   0, 131,   3,   0],
        [  3,  17,   0,   2,   2,   7,   0,   0, 134,   1],
        [  1,   1,   3,   1,   0,   5,   3,   2,   1, 247]]),
 'Precisión Global': 0.9098156452416543,
 'Error Global': 0.09018435475834574,
 'Precisión por categoría':        cero    cinco  cuatro       dos     nueve      ocho      seis  \
 0  0.955432  0.90625   0.895  0.883838  0.960452  0.873494  0.923529   
 
       siete      tres       uno  
 0  0.891156  0.807229  0.935606  }

In [114]:
dataTrainComprimido = comprimir(dataTrain, filtradoPorMayor=True)
dataTestComprimido = comprimir(dataTest, filtradoPorMayor=True)

modeloDataSinComprimir = MLPClassifier(hidden_layer_sizes =(250,100,50,25), max_iter = 50000, activation = 'relu', solver = 'adam',random_state=0)
analisisModelo2 = Analisis_Predictivo(predecir= "Numero",modelo=modeloDataSinComprimir, random_state=0, xtrain= dataTrainComprimido, xtest=dataTestComprimido, ytrain=ytrain, ytest=ytest)
analisisModelo2.fit_predict_resultados(inyectado=ytrain.append(ytest), custom=True)


Matriz de Confusión:
[[316   1   0  10   1   8  15   0   1   7]
 [  5 120   2   4   3   3   4   0  17   2]
 [  2   1 163   5  20   0   3   2   1   3]
 [  9   3   1 171   1   2   1   1   5   4]
 [  0   0  20   1 136   4   0   9   1   6]
 [ 11   8   4   4   6  95   4   1   8  25]
 [ 15   1   0   4   0   0 147   0   3   0]
 [  0   0   9   0  15   4   0 115   1   3]
 [  8  17   2   9   2   5   1   4 116   2]
 [  0   1   6   2   0   3   5   5   0 242]]

Precisión Global:
0.807673143996014

Error Global:
0.19232685600398602

Precisión por categoría:
       cero  cinco  cuatro       dos     nueve      ocho      seis     siete  \
0  0.880223   0.75   0.815  0.863636  0.768362  0.572289  0.864706  0.782313   

       tres       uno  
0  0.698795  0.916667  


{'Matriz de Confusión': array([[316,   1,   0,  10,   1,   8,  15,   0,   1,   7],
        [  5, 120,   2,   4,   3,   3,   4,   0,  17,   2],
        [  2,   1, 163,   5,  20,   0,   3,   2,   1,   3],
        [  9,   3,   1, 171,   1,   2,   1,   1,   5,   4],
        [  0,   0,  20,   1, 136,   4,   0,   9,   1,   6],
        [ 11,   8,   4,   4,   6,  95,   4,   1,   8,  25],
        [ 15,   1,   0,   4,   0,   0, 147,   0,   3,   0],
        [  0,   0,   9,   0,  15,   4,   0, 115,   1,   3],
        [  8,  17,   2,   9,   2,   5,   1,   4, 116,   2],
        [  0,   1,   6,   2,   0,   3,   5,   5,   0, 242]]),
 'Precisión Global': 0.807673143996014,
 'Error Global': 0.19232685600398602,
 'Precisión por categoría':        cero  cinco  cuatro       dos     nueve      ocho      seis     siete  \
 0  0.880223   0.75   0.815  0.863636  0.768362  0.572289  0.864706  0.782313   
 
        tres       uno  
 0  0.698795  0.916667  }

In [115]:
dataTrainComprimido = comprimir(dataTrain, filtradoPorMayor=True)
dataTestComprimido = comprimir(dataTest, filtradoPorMayor=True)

In [116]:
dataTrainComprimido

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-1.000,1.000,1.0,-1.000,0.909,1.0,0.996,0.867,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.864
1,0.392,1.000,1.0,1.000,0.366,1.0,1.000,0.653,1.000,0.214,0.993,1.000,1.000,1.000,1.000,1.000
2,-1.000,-0.888,1.0,-0.189,0.988,1.0,1.000,-0.977,1.000,1.000,1.000,1.000,-1.000,0.046,1.000,-1.000
3,0.409,1.000,1.0,1.000,1.000,1.0,1.000,0.695,-1.000,0.061,1.000,-1.000,-1.000,1.000,0.958,-1.000
4,-1.000,1.000,1.0,-1.000,-1.000,1.0,1.000,-0.757,0.337,0.224,0.998,1.000,1.000,1.000,1.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,0.167,1.000,1.0,-0.586,-1.000,1.0,1.000,-0.950,-0.318,1.000,1.000,0.219,0.233,1.000,1.000,-0.628
7287,-0.713,1.000,1.0,-1.000,-1.000,1.0,1.000,-1.000,-1.000,0.970,1.000,-0.589,-0.885,1.000,1.000,-1.000
7288,1.000,1.000,1.0,0.992,-0.685,1.0,1.000,0.566,0.754,1.000,1.000,1.000,1.000,1.000,1.000,0.860
7289,0.353,1.000,1.0,1.000,1.000,1.0,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [117]:
dataTestComprimido


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.810,1.000,1.00,1.000,1.00,0.248,1.000,1.000,0.955,1.000,1.0,-1.000,-1.000,-0.424,1.00,-1.000
1,-0.467,0.963,-1.00,-1.000,1.00,0.960,0.808,1.000,1.000,0.820,1.0,1.000,0.980,1.000,1.00,0.250
2,1.000,1.000,1.00,1.000,-1.00,1.000,1.000,1.000,-1.000,1.000,1.0,1.000,1.000,1.000,1.00,1.000
3,-0.883,0.999,-1.00,-1.000,1.00,1.000,1.000,1.000,1.000,0.179,1.0,1.000,1.000,1.000,1.00,0.177
4,-1.000,1.000,1.00,-1.000,1.00,1.000,-1.000,-1.000,1.000,0.999,1.0,1.000,1.000,1.000,1.00,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,-1.000,1.000,1.00,-1.000,-1.00,-0.797,1.000,-1.000,-1.000,-1.000,1.0,0.984,0.957,1.000,1.00,0.899
2003,-1.000,1.000,1.00,-0.175,-0.07,1.000,1.000,-0.859,-1.000,0.396,1.0,-1.000,-1.000,0.871,1.00,-1.000
2004,0.768,1.000,0.82,-1.000,1.00,1.000,1.000,1.000,1.000,1.000,1.0,-1.000,-1.000,1.000,0.06,-1.000
2005,1.000,1.000,1.00,0.645,1.00,1.000,1.000,1.000,1.000,1.000,1.0,1.000,1.000,1.000,1.00,0.753
